In [1]:
'''Bài 1. Sử dụng dữ liệu từ nguồn: https://www.kaggle.com/competitions/titanic/data
Yêu cầu:
+ Đối với cột age, thay vì chỉ điền giá trị trung bình hoặc trung vị, xây dựng một mô hình hồi quy tuyến tính
(Linear Regression) để dự đoán giá trị thiếu dựa trên các đặc trưng khác như pclass, sex, sibsp, và parch.
+ Đối với cột embarked, sử dụng thuật toán K-Nearest Neighbors (KNN) để điền giá trị thiếu dựa trên các đặc trưng liên quan.
+ Xử lý cột cabin (thường có nhiều giá trị thiếu): trích xuất thông tin từ ký tự đầu tiên của cabin (ví dụ: 'C', 'B') để tạo cột mới deck,
sau đó mã hóa cột này.
+ Tạo cột family_size như bài gốc (tổng sibsp + parch + 1).
+ Tạo cột is_alone (1 nếu hành khách đi một mình, 0 nếu đi cùng gia đình).
+ Trích xuất danh xưng (title) từ cột name (ví dụ: Mr, Mrs, Miss) và tạo cột title. Gộp các danh xưng hiếm (như Dr, Rev) thành nhóm "Rare".
+ Tạo cột fare_per_person bằng cách chia fare cho family_size để thể hiện giá vé trung bình mỗi người.
+ Vẽ biểu đồ boxplot để kiểm tra phân phối của fare_per_person theo pclass và survived.
+ Sử dụng kiểm định thống kê (ví dụ: t-test hoặc ANOVA) để kiểm tra xem fare_per_person có khác biệt đáng kể giữa các nhóm survived (sống sót hay không).
+ Xây dựng một pipeline xử lý dữ liệu (sử dụng sklearn.pipeline) để tự động hóa quá trình làm sạch, mã hóa, và chuẩn hóa dữ liệu.
+ Huấn luyện mô hình Random Forest để dự đoán survived dựa trên các đặc trưng đã xử lý và tạo mới.
+ Sử dụng Grid Search hoặc Random Search để tối ưu hóa siêu tham số của mô hình (ví dụ: n_estimators, max_depth).
+ Đánh giá mô hình bằng cross-validation (5-fold) và báo cáo các chỉ số: accuracy, precision, recall, F1-score, và ROC-AUC.
+ Tạo biểu đồ feature importance để xác định các đặc trưng quan trọng nhất ảnh hưởng đến khả năng sống sót.'''


'Bài 1. Sử dụng dữ liệu từ nguồn: https://www.kaggle.com/competitions/titanic/data\nYêu cầu:\n+ Đối với cột age, thay vì chỉ điền giá trị trung bình hoặc trung vị, xây dựng một mô hình hồi quy tuyến tính\n(Linear Regression) để dự đoán giá trị thiếu dựa trên các đặc trưng khác như pclass, sex, sibsp, và parch.\n+ Đối với cột embarked, sử dụng thuật toán K-Nearest Neighbors (KNN) để điền giá trị thiếu dựa trên các đặc trưng liên quan.\n+ Xử lý cột cabin (thường có nhiều giá trị thiếu): trích xuất thông tin từ ký tự đầu tiên của cabin (ví dụ: \'C\', \'B\') để tạo cột mới deck,\nsau đó mã hóa cột này.\n+ Tạo cột family_size như bài gốc (tổng sibsp + parch + 1).\n+ Tạo cột is_alone (1 nếu hành khách đi một mình, 0 nếu đi cùng gia đình).\n+ Trích xuất danh xưng (title) từ cột name (ví dụ: Mr, Mrs, Miss) và tạo cột title. Gộp các danh xưng hiếm (như Dr, Rev) thành nhóm "Rare".\n+ Tạo cột fare_per_person bằng cách chia fare cho family_size để thể hiện giá vé trung bình mỗi người.\n+ Vẽ biểu đ

In [4]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder

train_data = pd.read_csv(r'train.csv')
test_data = pd.read_csv(r'test.csv')

In [ ]:
# 1. Đối với cột age, thay vì chỉ điền giá trị trung bình hoặc trung vị, xây dựng một mô hình hồi quy tuyến tính
# (Linear Regression) để dự đoán giá trị thiếu dựa trên các đặc trưng khác như pclass, sex, sibsp, và parch.
features = ['Age', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']
data = train_data[features].copy()

data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})

known_age = data[data['Age'].notna()]
unknown_age = data[data['Age'].isna()]

X = known_age.drop('Age', axis = 1)
y = known_age['Age']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'Model score (R^2): {model.score(X_test, y_test): .2f}')

if not unknown_age.empty:
    X_missing = unknown_age.drop('Age', axis = 1)
    predicted_age = model.predict(X_missing)
    train_data.loc[train_data['Age'].isna(), 'Age'] = predicted_age

print(f"Số giá trị còn thiếu trong Age: {train_data['Age'].isna().sum}")

Mean Squared Error (Age): 140.71
Model score (R^2) for Age: 0.24
Số giá trị còn thiếu trong Age: 0
Số giá trị thiếu trong cột Embarked sau khi điền: 0


In [11]:
#2. Đối với cột embarked, sử dụng thuật toán K-Nearest Neighbors (KNN) để điền giá trị thiếu dựa trên các đặc trưng liên quan.

# Đọc dữ liệu
train_data = pd.read_csv('train.csv')

# Chọn các đặc trưng liên quan
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
data = train_data[features].copy()

# Chuyển đổi biến phân loại thành số
data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})

# Mã hóa Embarked (nếu cần)
le = LabelEncoder()
data['Embarked_encoded'] = le.fit_transform(data['Embarked'].astype(str))

# Tách dữ liệu
known_embarked = data[data['Embarked'].notna()]
unknown_embarked = data[data['Embarked'].isna()]

# Chuẩn bị dữ liệu huấn luyện
X_train = known_embarked.drop(['Embarked', 'Embarked_encoded'], axis=1)
y_train = known_embarked['Embarked_encoded']

# Chuẩn hóa dữ liệu (quan trọng với KNN)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

if not unknown_embarked.empty:
    X_test = unknown_embarked.drop(['Embarked', 'Embarked_encoded'], axis=1)
    X_test_scaled = scaler.transform(X_test)
    
    # Dự đoán
    predicted_embarked = knn.predict(X_test_scaled)
    
    # Chuyển đổi ngược về giá trị gốc
    predicted_embarked_labels = le.inverse_transform(predicted_embarked)
    
    # Điền giá trị dự đoán vào dữ liệu gốc
    train_data.loc[train_data['Embarked'].isna(), 'Embarked'] = predicted_embarked_labels

# Kiểm tra số lượng giá trị thiếu còn lại
print(f"Số giá trị thiếu trong cột Embarked sau khi điền: {train_data['Embarked'].isna().sum()}")

Số giá trị thiếu trong cột Embarked sau khi điền: 0


In [ ]:
'''3. Xử lý cột cabin (thường có nhiều giá trị thiếu): trích xuất thông tin từ ký tự đầu tiên của cabin (ví dụ: 'C', 'B') để tạo cột mới deck,
sau đó mã hóa cột này.'''
train_data['Deck'] = train_data['Cabin'].str[0]
print('Phân phối các Deck trước khi xử lý')
print(train_data['Deck'].value_counts(dropna=False))

Deck
NaN    687
C       59
B       47
D       33
E       32
A       15
F       13
G        4
T        1
Name: count, dtype: int64
